<a href="https://colab.research.google.com/github/fjadidi2001/AD_Prediction/blob/main/AD_Prediction_Mar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from graphviz import Digraph

# Create a flowchart
flowchart = Digraph("Alzheimer Prediction", format="png")

# Start
flowchart.node("Start", "Start", shape="oval", style="filled", fillcolor="lightblue")

# Data Collection
flowchart.node("Data", "Collect Voice Dataset\n(ADReSS, ADReSSo, Pitt Corpus)", shape="parallelogram")

# Preprocessing
flowchart.node("Preprocessing", "Preprocessing\n- Convert audio to text (ASR)\n- Extract Acoustic & Linguistic Features", shape="box")

# Feature Extraction
flowchart.node("Feature Extraction", "Feature Extraction\n- Audio: Wav2Vec2, HuBERT, CNN\n- Text: BERT, RoBERTa", shape="box")

# Model Selection
flowchart.node("Model Selection", "Select Best Model\nMultimodal: CNN + BERT\nFusion: Tensor Fusion, Score Fusion", shape="diamond")

# Training
flowchart.node("Training", "Train Model\n- Fine-tune Pretrained Models\n- Apply Cross-Validation", shape="box")

# Prediction
flowchart.node("Prediction", "Predict Outcome\n- Alzheimer's (AD) or Healthy (HC)\n- MMSE Score (Regression Task)", shape="parallelogram")

# Evaluation
flowchart.node("Evaluation", "Evaluate Performance\n(Accuracy, F1-score, RMSE)", shape="box")

# Deployment
flowchart.node("Deployment", "Deploy Model\n(Clinical Tool / App)", shape="parallelogram")

# End
flowchart.node("End", "End", shape="oval", style="filled", fillcolor="lightblue")

# Connect nodes
flowchart.edge("Start", "Data")
flowchart.edge("Data", "Preprocessing")
flowchart.edge("Preprocessing", "Feature Extraction")
flowchart.edge("Feature Extraction", "Model Selection")
flowchart.edge("Model Selection", "Training")
flowchart.edge("Training", "Prediction")
flowchart.edge("Prediction", "Evaluation")
flowchart.edge("Evaluation", "Deployment")
flowchart.edge("Deployment", "End")

# Render the flowchart
flowchart_path = "/content/data/alzheimer_prediction_flowchart"
flowchart.render(flowchart_path)

# Return the path to the generated flowchart image
flowchart_path + ".png"

'/content/data/alzheimer_prediction_flowchart.png'